In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import linear_model


In [2]:
#define file
file = pd.read_csv('marketing_customer_analysis.csv')

In [26]:
#changed date into a better format
file['Effective To Date'] = pd.to_datetime(file['Effective To Date'])
#display(file.shape)
#file.head()

In [5]:
#separate the features from the labels
y = file['Total Claim Amount']
X = file.drop(['Total Claim Amount'], axis=1)

In [27]:
#X.head()

In [7]:
#categorical features and numerical ones are going to be treated differently
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [28]:
#X_cat.head()

In [29]:
#X_num

In [30]:
from sklearn.preprocessing import MinMaxScaler
MinMaxtransformer = MinMaxScaler().fit(X_num)
x_normalized = MinMaxtransformer.transform(X_num)
#print(x_normalized.shape)
x_normalized = pd.DataFrame(x_normalized,columns=X_num.columns)
#x_normalized.head()

In [31]:
#categorical values
X_cat = file.select_dtypes(include = object)
#X_cat.head()

In [32]:
#Unique detection before drop
#for col in X_cat.columns:
#    print(col + ":" + str(X_cat[col].nunique()))

In [14]:
X_cat_new = X_cat.drop('Customer',axis=1)

In [34]:
#Unique detection after drop
#display(X_cat_new.shape)
#X_cat_new.head()

In [35]:
#for col in X_cat_new.columns:
#    print(col + ":" + str(X_cat_new[col].nunique()))


In [84]:
#one hot encoder without a loop------------------
#from sklearn.preprocessing import OneHotEncoder
#encoder = OneHotEncoder(drop='first').fit(X_cat_new)   #dropped first column
#print(encoder.categories_)
#encoded = encoder.transform(X_cat_new).toarray()
#print(encoded)
#onehot_encoded = pd.DataFrame(encoded)
#onehot_encoded.head()

from sklearn.preprocessing import OneHotEncoder
df = pd.DataFrame()
for col in X_cat_new.columns:
    encoder = OneHotEncoder().fit(X_cat_new[[col]])
    #print(encoder.categories_)
    encoded = encoder.transform(X_cat_new[[col]]).toarray()
    #print(encoded)
    onehot_encoded = pd.DataFrame(encoded,columns=encoder.categories_)
    #onehot_encoded.head()
    df=pd.concat([df,onehot_encoded], axis=1)
df.head()


,Arizona,California,Nevada,Oregon,Washington,No,Yes,Basic,Extended,Premium,...,Web,Four-Door Car,Luxury Car,Luxury SUV,SUV,Sports Car,Two-Door Car,Large,Medsize,Small
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [85]:
#label encoding 
from sklearn.preprocessing import LabelEncoder
label_encoded = pd.DataFrame(data=None, columns=X_cat_new.columns)
for col in X_cat_new:
    label_encoded[col] = LabelEncoder().fit(X_cat_new[col]).transform(X_cat_new[col]) 
display(label_encoded.head())

,State,Response,Coverage,Education,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size
0,4,0,0,0,1,0,1,1,0,2,0,0,5,1
1,0,0,1,0,4,0,1,2,1,5,2,0,0,1
2,2,0,2,0,1,0,1,1,1,5,0,0,5,1
3,1,0,0,0,4,1,1,1,0,1,0,2,3,1
4,4,0,0,0,1,1,0,2,1,3,0,0,0,1


In [45]:
X = pd.concat([x_normalized, onehot_encoded], axis=1)
#y  is already defined

In [46]:
X.shape

(9134, 50)

In [50]:
#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [51]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7307, 50)
(1827, 50)
(7307,)
(1827,)


In [61]:
#X_train.head()

In [62]:
#y_train.head()

In [86]:
#train/fit model 
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [87]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.7791817833865978

In [88]:
#evaluate test data
predictions = lm.predict(X_test)
r2_score(y_test, predictions)

0.7476579101411098

In [89]:
#y_test

In [73]:
predictions

array([ 445.49861591,  410.95187025,  470.82353292, ..., 1097.96599736,
        755.09512839,   94.70529522])

In [66]:
from sklearn.metrics import mean_squared_error       #show the mean squared error
np.sqrt(mean_squared_error(y_test,predictions))

156.32820708183144

In [67]:
#metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [68]:
mae = mean_absolute_error(y_test, predictions)
print(mae)

100.77835320884411


In [71]:
mse = mean_squared_error(y_test,predictions)
print(mse)

24438.508329419976


In [72]:
rmse = np.sqrt(mse)
print(rmse)

156.32820708183144


In [70]:
r2 = r2_score(y_test, predictions)
r2

0.7476579101411098